# Imports

In [ ]:
import pandas as pd
import numpy as np
import glob
import re
import datetime as dt
import ollama
import json
from tqdm import tqdm
import time
import os


# display options
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 
from IPython.display import display
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# CFPB

In [3]:

df = pd.read_csv("Datasets\CFPB_Complaints.csv")
df.head()
df.shape

df_raw = df


,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2020-07-06,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,FL,346XX,NaN,Other,Web,2020-07-06,Closed with explanation,Yes,NaN,3730948
1,2025-10-06,Credit reporting or other personal consumer re...,Credit reporting,Problem with a company's investigation into an...,Investigation took more than 30 days,NaN,NaN,Experian Information Solutions Inc.,TX,75454,NaN,NaN,Web,2025-10-06,In progress,Yes,NaN,16372503
2,2025-10-14,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Information is missing that should be on the r...,NaN,NaN,"EQUIFAX, INC.",TX,75062,NaN,NaN,Web,2025-10-14,In progress,Yes,NaN,16558024
3,2025-10-26,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,NaN,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",CA,93619,NaN,NaN,Web,2025-10-26,In progress,Yes,NaN,16826857
4,2025-10-10,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,NaN,"EQUIFAX, INC.",GA,30341,NaN,NaN,Web,2025-10-10,In progress,Yes,NaN,16507707


(12126091, 18)

# Exploratory Analysis

## df_sub

In [4]:
# subset to the 2024 complaints for selected financial institutions 

# descriptive analysis
df = df_raw


# get available comments
df = df[~df['Consumer complaint narrative'].isna()][~df['Consumer complaint narrative'].isin(['NA','N/A','Not Available',''])]


# narrow down to 2024 data
df['Date received'] = pd.to_datetime(df['Date received'])
df = df[df['Date received']>='2024-01-01']
df.head(2)
df.shape


# subset to specific financial institutions
    # Chase, Bank of America, Wells Fargo, PNC, JPMorgan, Discover, Capital One, Vanguard, and Charles Schwab.

df = df[df['Company'].str.contains(r'Chase|Bank of America|Fargo|PNC|JPMorgan|Discover BANK|Capital One|Vanguard|Schwab',case = False)]


df.head(2)
df.shape
df_sub = df



C:\Users\irene\AppData\Local\Temp\ipykernel_25896\2210883069.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[~df['Consumer complaint narrative'].isna()][~df['Consumer complaint narrative'].isin(['NA','N/A','Not Available',''])]


,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
26,2024-01-05,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Information belongs to someone else,Kindly address this issue on my credit report....,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,IL,60502,NaN,Consent provided,Web,2024-01-05,Closed with non-monetary relief,Yes,NaN,8113747
32,2025-09-29,Credit reporting or other personal consumer re...,Credit reporting,Improper use of your report,Reporting company used your report improperly,I am formally disputing the accuracy of the fo...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",NC,27545,NaN,Consent provided,Web,2025-09-29,Closed with explanation,Yes,NaN,16237002


(1830636, 18)

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
39,2025-10-29,Checking or savings account,Checking account,Managing an account,Problem using a debit or ATM card,"On XX/XX/XXXX, a {$890.00} charge was taken fr...",Company has responded to the consumer and the ...,WELLS FARGO & COMPANY,AZ,85251,NaN,Consent provided,Web,2025-10-29,Closed with explanation,Yes,NaN,16899880
275,2025-08-15,Credit reporting or other personal consumer re...,Credit reporting,Improper use of your report,Credit inquiries on your report that you don't...,"On XX/XX/scrub>, 2025 I received a text saying...",NaN,CAPITAL ONE FINANCIAL CORPORATION,IN,463XX,NaN,Consent provided,Web,2025-08-15,Closed with non-monetary relief,Yes,NaN,15305109


(80941, 18)

## sample complaints

In [5]:
# sample complaints
df = df_sub

df_f = df[df.Company.str.contains('Bank of America', case=False)] # fidelity Vanguard
df_f.shape
df_f.sample(1)['Consumer complaint narrative'].to_list()


(14269, 18)

['I have a problem with Bank of America. They are liars. They are thieves and they are crooks. They will deny a legitimate fraudulent transaction. I filed a dispute for the amount {$970.00} that was fraudulent debited from my account after I came back from my trip in XXXX. I called immediately and the initial rep stated they know and can see that I did not physically use my card. So if I did not physically, use my card and you can check the records how was my card use when I was asleep in my bed that does not make any sense and you guys Bank of America denied my dispute and tell me theyre going to purposely overdraw my account for the credit that they provided me for {$970.00}. That is outrageous. Ive been with Bank of America for almost 20 years. I want this matter solved. This is unreasonable, unfair, and unstable.']

## type of products, % dispute, etc.

In [6]:
df = df_sub


# types of the products

set(df.Product)
set(df['Sub-product'])

# types of the complaints
set(df.Issue)
set(df['Sub-issue'])

# companies
set(df.Company)


# what % gets the timely response of the company

df_t = df.groupby(['Timely response?']).size().reset_index()
df_t=df_t.rename(columns={0:'perct'})
# df_t
df_t['perct'] = df_t['perct']/len(df)*100
df_t


# sample complaints
df.sample(5)['Consumer complaint narrative'].to_list()





{'Checking or savings account',
 'Credit card',
 'Credit reporting or other personal consumer reports',
 'Debt collection',
 'Debt or credit management',
 'Money transfer, virtual currency, or money service',
 'Mortgage',
 'Payday loan, title loan, personal loan, or advance loan',
 'Prepaid card',
 'Student loan',
 'Vehicle loan or lease'}

{'Auto debt',
 'CD (Certificate of Deposit)',
 'Check cashing service',
 'Checking account',
 'Conventional home mortgage',
 'Credit card debt',
 'Credit repair services',
 'Credit reporting',
 'Debt settlement',
 'Domestic (US) money transfer',
 'Earned wage access',
 'FHA mortgage',
 'Federal student loan debt',
 'Federal student loan servicing',
 'Foreign currency exchange',
 'General-purpose credit card or charge card',
 'General-purpose prepaid card',
 'Gift card',
 'Government benefit card',
 'Home equity loan or line of credit (HELOC)',
 'I do not know',
 'Installment loan',
 'International money transfer',
 'Lease',
 'Loan',
 'Manufactured home loan',
 'Medical debt',
 'Mobile or digital wallet',
 "Money order, traveler's check or cashier's check",
 'Mortgage debt',
 'Mortgage modification or foreclosure avoidance',
 'Other advances of future income',
 'Other banking product or service',
 'Other debt',
 'Other personal consumer report',
 'Other type of mortgage',
 'Payday loan'

{'Advertising',
 'Advertising and marketing, including promotional offers',
 'Applying for a mortgage or refinancing an existing mortgage',
 'Attempts to collect debt not owed',
 "Can't contact lender or servicer",
 "Charged fees or interest you didn't expect",
 'Charged upfront or unexpected fees',
 'Closing an account',
 'Closing on a mortgage',
 'Closing your account',
 'Communication tactics',
 'Confusing or misleading advertising or marketing',
 'Confusing or missing disclosures',
 'Credit limit changed',
 'Credit monitoring or identity theft protection services',
 'Dealing with your lender or servicer',
 "Didn't provide services promised",
 'Electronic communications',
 'False statements or representation',
 'Fees or interest',
 'Fraud or scam',
 'Getting a credit card',
 'Getting a line of credit',
 'Getting a loan',
 'Getting a loan or lease',
 'Getting the loan',
 'Identity theft protection or other monitoring services',
 'Improper use of your report',
 'Incorrect exchange rat

{'Account information incorrect',
 'Account opened without my consent or knowledge',
 'Account reinstatement or redemption after repossession',
 'Account sold or transferred to another company',
 'Account status incorrect',
 'Add-on products and services',
 'An existing modification, forbearance plan, short sale, or other loss mitigation relief',
 'Application denials',
 'Application denied',
 'Applying for or obtaining a modification, forbearance plan, short sale, or deed-in-lieu',
 'Attempted to collect wrong amount',
 'Banking errors',
 'Bankruptcy',
 'Billing dispute for services',
 'Billing or statement issues',
 'Billing problem',
 'Bounced checks or returned payments',
 'Called before 8am or after 9pm',
 "Can't close your account",
 "Can't get other flexible options for repaying your loan",
 "Can't stop withdrawals from your account",
 "Can't temporarily delay making payments",
 "Can't use card to make purchases",
 'Cannot graduate, receive diploma, or get transcript due to mone

{'BANK OF AMERICA, NATIONAL ASSOCIATION',
 'CAPITAL ONE FINANCIAL CORPORATION',
 'CHARLES SCHWAB CORPORATION, THE',
 'DISCOVER BANK',
 'JPMORGAN CHASE & CO.',
 'PNC Bank N.A.',
 'Vanguard Financial Services, Inc.',
 'WELLS FARGO & COMPANY'}

,Timely response?,perct
0,No,0.156904
1,Yes,99.843096


['Please note the above referenced accounts were opened FRAUDULENTLY ; as a result of Identity Theft. My information was compromised & fraudulently used without my permission. My name, social security number, along with fake addresses, emails & phone numbers were also fraudulently used by the fraudster ( s ).',
 'On approximately XX/XX/XXXX, XXXX XXXX XXXX issued me a credit/refund for ticket XXXX in the amount of {$240.00}. This went to a credit card ending in XXXX. \n\nI dont have a credit card ending in XXXX, but the original purchase was made with one of my Chase credit cards using XXXX XXXX, so I believe the XXXX to be the last four of a temporary virtual card number. \n\nThe refund never posted to my account, so I followed up with XXXX multiple times. On XX/XX/XXXX, they again confirmed the refund went through and this time provided me with an ARN ( Acquirer Reference Number ) that I understand proves that the refund went through. The ARN is XXXX. \n\nI reached out to Chase with 

# Data Sampling

In [7]:

# get a sample of 250
df = df_sub

df = df.sample(100)

df.to_csv('./Datasets/CFPB_sample.csv',index=False)

# LLM

## parameters - system prompt & model name


In [3]:


SYSTEM_PROMPT = """You are an expert financial risk analyst. Your task is to score a CFPB consumer complaint based on three criteria: Severity, Complexity, and Actionability. Always output your assessment as a single, valid JSON object.

SCORING CRITERIA (Scale 1 to 5):

1. SEVERITY (1-5): Measures the magnitude of harm or potential harm to the consumer.
   - 1: Minor inconvenience, easily reversible, zero to minimal financial loss (<$100).
   - 5: Catastrophic financial loss (>$10,000), permanent damage to credit/housing, or systemic risk of failure.

2. COMPLEXITY (1-5): Measures the difficulty of investigation and resolution.
   - 1: Simple error (e.g., incorrect fee), clear-cut facts, easily corrected.
   - 5: Legal ambiguity, involves multiple third parties (e.g., collector, servicer, original lender), requires extensive document review, or suggests a novel systemic issue.

3. ACTIONABILITY (1-5): Measures the quality, substance, and utility of the complaint narrative for regulatory action.
   - 1: Low Actionability: Vague, extremely short, contains general accusations but lacks specific dates, dollar amounts, account numbers, or company names. Cannot be acted upon without significant follow-up.
   - 5: High Actionability: Long, detailed narrative (multiple paragraphs), provides sufficient dates, specific dollar amounts, account information, and is written clearly and professionally. Provides sufficient basis for immediate investigation.

Output ONLY a JSON object with the keys 'severity', 'complexity', and 'actionability'.
"""

MODEL_NAME = 'llama3'

OLLAMA_HOST = 'http://localhost:11434' # Default local endpoint

## API programming - function


In [4]:

# Initialize Ollama Client
client = ollama.Client(host=OLLAMA_HOST)


def score_complaint(complaint_text: str, model_name: str = MODEL_NAME):
    """Sends a single complaint to the local Ollama API for structured scoring."""
    
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"Score the following complaint: {complaint_text}"}
    ]
    
    try:
        # Call the local Ollama API
        response = client.chat(
            model=model_name,
            messages=messages,
            format="json",  # Forces JSON output from the model
            stream=False,
            # Lower temperature for deterministic, consistent scoring
            options={'num_gpu': 0, 'num_thread': 4, 'temperature': 0.1} 
        )
        
        # The JSON content is returned as a string in the 'content' key
        json_string = response['message']['content']
        scores = json.loads(json_string)
        
        # Basic validation to ensure all required keys are present
        if all(key in scores for key in ['severity', 'complexity', 'actionability']):
            return scores
        else:
            raise ValueError("Missing required keys in JSON output.")
        
    except Exception as e:
        # Log any errors (API connection issue, invalid JSON, key error)
        print(f"Error processing complaint: {e}")
        return {'severity': 0, 'complexity': 0, 'actionability': 0, 'error_detail': str(e)}



##  Loop Through All Complaints

In [8]:

# Read the CSV file
df = pd.read_csv('./Datasets/CFPB_sample.csv')

# Create new columns for the scores
df['severity'] = 0
df['complexity'] = 0
df['actionability'] = 0
df['error_detail'] = None

# Create a directory for batch files if it doesn't exist
batch_dir = './Datasets/batches/'
os.makedirs(batch_dir, exist_ok=True)

# Initialize variables for batch processing
batch_size = 10
batch_number = 1
batch_results = []

print(f"Starting processing of {len(df)} complaints...")
print(f"Saving batches of {batch_size} records to '{batch_dir}'")

# Use tqdm for progress tracking
for index, row in tqdm(df.iterrows(), total=len(df)):
    
    scores = score_complaint(row['Consumer complaint narrative'])
    
    # Store the results, using .get() for safety
    df.loc[index, 'severity'] = scores.get('severity', 0)
    df.loc[index, 'complexity'] = scores.get('complexity', 0)
    df.loc[index, 'actionability'] = scores.get('actionability', 0)
    df.loc[index, 'error_detail'] = scores.get('error_detail', None)
    
    # Add this row to batch results
    batch_results.append({
        'index': index,
        'Complaint ID': row['Complaint ID'],
        'Consumer complaint narrative': row['Consumer complaint narrative'],
        'severity': scores.get('severity', 0),
        'complexity': scores.get('complexity', 0),
        'actionability': scores.get('actionability', 0),
        'error_detail': scores.get('error_detail', None)
    })
    
    # Save batch every 50 records
    if len(batch_results) >= batch_size:
        batch_df = pd.DataFrame(batch_results)
        batch_file = f'{batch_dir}batch_{batch_number:03d}.csv'
        batch_df.to_csv(batch_file, index=False)
        
        print(f"Saved batch {batch_number} to {batch_file}")
        
        # Reset for next batch
        batch_results = []
        batch_number += 1
    
    # Optional: Wait briefly to reduce CPU load/heat
    time.sleep(0.1)

#Save the last batch (if any remaining records)
if batch_results:
    batch_df = pd.DataFrame(batch_results)
    batch_file = f'{batch_dir}batch_{batch_number:03d}.csv'
    batch_df.to_csv(batch_file, index=False)
    print(f"Saved final batch {batch_number} to {batch_file}")

# Now save the complete results
print("\nProcessing complete. Saving final results...")

# Display Results
print("\n--- Final Scored Data Sample ---")
df[['Consumer complaint narrative', 'severity', 'complexity', 'actionability', 'error_detail']].head()

# Save the complete dataset
df.to_csv('./Datasets/Metrics.csv', index=False)
print(f"\nComplete dataset saved to ./Datasets/Metrics.csv")

# Optional: Combine all batch files into one for verification
print("\n--- Batch Files Summary ---")
batch_files = sorted([f for f in os.listdir(batch_dir) if f.endswith('.csv')])
for batch_file in batch_files:
    batch_path = os.path.join(batch_dir, batch_file)
    batch_df = pd.read_csv(batch_path)
    print(f"{batch_file}: {len(batch_df)} records")

Starting processing of 100 complaints...
Saving batches of 10 records to './Datasets/batches/'


 10%|█         | 10/100 [05:31<40:05, 26.72s/it] 

Saved batch 1 to ./Datasets/batches/batch_001.csv


 20%|██        | 20/100 [08:06<22:52, 17.16s/it]

Saved batch 2 to ./Datasets/batches/batch_002.csv


...

Saved batch 9 to ./Datasets/batches/batch_009.csv


100%|██████████| 100/100 [35:50<00:00, 21.50s/it]

Saved batch 10 to ./Datasets/batches/batch_010.csv

Processing complete. Saving final results...

--- Final Scored Data Sample ---


,Consumer complaint narrative,severity,complexity,actionability,error_detail
0,XXXX XXXX XXXX was hacked and my personal info...,5,4,5,None
1,On XX/XX/year> I went to a Kohl 's store in XX...,4,3,5,None
2,On XX/XX/XXXX I deposited a {$25000.00} verifi...,5,4,5,None
3,I charged ~ {$660.00} on XX/XX/XXXX using my X...,4,3,5,None
4,"A few weeks ago, a friends of mine had clients...",4,3,5,None



Complete dataset saved to ./Datasets/Metrics.csv

--- Batch Files Summary ---
batch_001.csv: 10 records
batch_002.csv: 10 records
batch_003.csv: 10 records
batch_004.csv: 10 records
batch_005.csv: 10 records
batch_006.csv: 10 records
batch_007.csv: 10 records
batch_008.csv: 10 records
batch_009.csv: 10 records
batch_010.csv: 10 records


# Metrics

In [36]:
df = pd.read_csv('./Datasets/Metrics.csv')
df.head(2)
df.shape
df_0 = df


# check company distribution
df.groupby(['Company']).size().reset_index()
df.groupby(['Product']).size().reset_index()


# by company 
df = df_0


# get % of records by company
df_p =  df.groupby(['Company'])[[ 'Complaint ID']].size().reset_index()
df_p=df_p.rename(columns={0:'perct'})
df_p['perct'] = df_p['perct']/len(df)
df_p['perct'] = df_p['perct']*5 # normalize to 0-5
df_p


# avg severity, complexity, actionability by company & product
df_1 = df.groupby(['Company'])[[ 'severity', 'complexity', 'actionability']].mean().reset_index()
df_1

# avg delayed response by company & product
df['delay'] = df['Timely response?'].apply(lambda x:1 if x=="No" else 0)
df_2 = df.groupby(['Company'])['delay'].mean().reset_index()
df_2

# merge the avg scores
df = df_p.merge(df_1, how = 'left', on = ['Company']).merge(df_2, how = 'left', on = ['Company'])


# final scores: weighted sum 
    # here the delay is assigned a very low weight due to majority of the records got a timely response
    # total weight is 1
df['Final Risk Score'] = 0.3 * df['perct']  +  0.2 * df['severity'] +  0.2 * df['complexity']+  0.2 * df['actionability']+ 0.1 * df['delay']
df['Final Risk Score'] = df['Final Risk Score'].round(2)


df.sort_values(by = 'Final Risk Score',ascending=False)

df.to_csv('./Datasets/ScoreCard_Company Level.csv',index=False)





# by company and product
df = df_0


# get % of records by company
df_p =  df.groupby(['Company','Product'])[[ 'Complaint ID']].size().reset_index()
df_p=df_p.rename(columns={0:'perct'})
df_p['perct'] = df_p['perct']/len(df)
df_p['perct'] = df_p['perct']*5 # normalize to 0-5
df_p


# avg severity, complexity, actionability by company & product
df_1 = df.groupby(['Company','Product'])[[ 'severity', 'complexity', 'actionability']].mean().reset_index()
df_1

# avg delayed response by company & product
df['delay'] = df['Timely response?'].apply(lambda x:1 if x=="No" else 0)
df_2 = df.groupby(['Company','Product'])['delay'].mean().reset_index()
df_2

# merge the avg scores
df = df_p.merge(df_1, how = 'left', on = ['Company','Product']).merge(df_2, how = 'left', on = ['Company','Product'])


# final scores: weighted sum 
    # here the delay is assigned a very low weight due to majority of the records got a timely response
    # total weight is 1
df['Final Risk Score'] = 0.3 * df['perct']  +  0.2 * df['severity'] +  0.2 * df['complexity']+  0.2 * df['actionability']+ 0.1 * df['delay']
df['Final Risk Score'] = df['Final Risk Score'].round(2)



df.sort_values(by = 'Final Risk Score',ascending=False)

df.to_csv('./Datasets/ScoreCard_Company & Product Level.csv',index=False)


,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,...,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,severity,complexity,actionability,error_detail
0,2024-02-27,Credit card,Store credit card,Getting a credit card,Card opened without my consent or knowledge,XXXX XXXX XXXX was hacked and my personal info...,NaN,CAPITAL ONE FINANCIAL CORPORATION,NC,28152,...,Web,2024-02-27,Closed with explanation,Yes,NaN,8423056,5,4,5,NaN
1,2024-06-27,Credit card,Store credit card,Problem with a purchase shown on your statement,Credit card company isn't resolving a dispute ...,On XX/XX/year> I went to a Kohl 's store in XX...,NaN,CAPITAL ONE FINANCIAL CORPORATION,ID,83713,...,Web,2024-06-27,Closed with explanation,Yes,NaN,9359274,4,3,5,NaN


(100, 22)

,Company,0
0,"BANK OF AMERICA, NATIONAL ASSOCIATION",17
1,CAPITAL ONE FINANCIAL CORPORATION,42
2,DISCOVER BANK,8
3,JPMORGAN CHASE & CO.,21
4,PNC Bank N.A.,1
5,WELLS FARGO & COMPANY,11


,Product,0
0,Checking or savings account,29
1,Credit card,35
2,Credit reporting or other personal consumer re...,24
3,Debt collection,4
4,"Money transfer, virtual currency, or money ser...",4
5,Mortgage,1
6,Student loan,1
7,Vehicle loan or lease,2


,Company,perct
0,"BANK OF AMERICA, NATIONAL ASSOCIATION",0.85
1,CAPITAL ONE FINANCIAL CORPORATION,2.10
2,DISCOVER BANK,0.40
3,JPMORGAN CHASE & CO.,1.05
4,PNC Bank N.A.,0.05
5,WELLS FARGO & COMPANY,0.55


...

,Company,perct,severity,complexity,actionability,delay,Final Risk Score
1,CAPITAL ONE FINANCIAL CORPORATION,2.10,3.476190,3.261905,4.642857,0.0,2.91
5,WELLS FARGO & COMPANY,0.55,4.090909,3.727273,5.000000,0.0,2.73
3,JPMORGAN CHASE & CO.,1.05,3.904762,3.190476,4.761905,0.0,2.69
2,DISCOVER BANK,0.40,4.000000,3.375000,4.875000,0.0,2.57
0,"BANK OF AMERICA, NATIONAL ASSOCIATION",0.85,3.823529,3.176471,4.470588,0.0,2.55
4,PNC Bank N.A.,0.05,4.000000,3.000000,5.000000,0.0,2.42


,Company,Product,perct
0,"BANK OF AMERICA, NATIONAL ASSOCIATION",Checking or savings account,0.35
1,"BANK OF AMERICA, NATIONAL ASSOCIATION",Credit card,0.35
2,"BANK OF AMERICA, NATIONAL ASSOCIATION",Credit reporting or other personal consumer re...,0.05
3,"BANK OF AMERICA, NATIONAL ASSOCIATION","Money transfer, virtual currency, or money ser...",0.10
4,CAPITAL ONE FINANCIAL CORPORATION,Checking or savings account,0.40
5,CAPITAL ONE FINANCIAL CORPORATION,Credit card,0.70
6,CAPITAL ONE FINANCIAL CORPORATION,Credit reporting or other personal consumer re...,0.80
7,CAPITAL ONE FINANCIAL CORPORATION,Debt collection,0.05
8,CAPITAL ONE FINANCIAL CORPORATION,"Money transfer, virtual currency, or money ser...",0.05
9,CAPITAL ONE FINANCIAL CORPORATION,Vehicle loan or lease,0.10


,Company,Product,severity,complexity,actionability
0,"BANK OF AMERICA, NATIONAL ASSOCIATION",Checking or savings account,4.142857,3.142857,4.857143
1,"BANK OF AMERICA, NATIONAL ASSOCIATION",Credit card,3.714286,3.571429,4.285714
2,"BANK OF AMERICA, NATIONAL ASSOCIATION",Credit reporting or other personal consumer re...,3.000000,2.000000,2.000000
3,"BANK OF AMERICA, NATIONAL ASSOCIATION","Money transfer, virtual currency, or money ser...",3.500000,2.500000,5.000000
4,CAPITAL ONE FINANCIAL CORPORATION,Checking or savings account,3.500000,3.500000,4.875000
5,CAPITAL ONE FINANCIAL CORPORATION,Credit card,3.642857,3.214286,4.785714
6,CAPITAL ONE FINANCIAL CORPORATION,Credit reporting or other personal consumer re...,3.312500,3.312500,4.812500
7,CAPITAL ONE FINANCIAL CORPORATION,Debt collection,4.000000,3.000000,2.000000
8,CAPITAL ONE FINANCIAL CORPORATION,"Money transfer, virtual currency, or money ser...",5.000000,4.000000,5.000000
9,CAPITAL ONE FINANCIAL CORPORATION,Vehicle loan or lease,2.500000,2.000000,2.500000


,Company,Product,delay
0,"BANK OF AMERICA, NATIONAL ASSOCIATION",Checking or savings account,0.0
1,"BANK OF AMERICA, NATIONAL ASSOCIATION",Credit card,0.0
2,"BANK OF AMERICA, NATIONAL ASSOCIATION",Credit reporting or other personal consumer re...,0.0
3,"BANK OF AMERICA, NATIONAL ASSOCIATION","Money transfer, virtual currency, or money ser...",0.0
4,CAPITAL ONE FINANCIAL CORPORATION,Checking or savings account,0.0
5,CAPITAL ONE FINANCIAL CORPORATION,Credit card,0.0
6,CAPITAL ONE FINANCIAL CORPORATION,Credit reporting or other personal consumer re...,0.0
7,CAPITAL ONE FINANCIAL CORPORATION,Debt collection,0.0
8,CAPITAL ONE FINANCIAL CORPORATION,"Money transfer, virtual currency, or money ser...",0.0
9,CAPITAL ONE FINANCIAL CORPORATION,Vehicle loan or lease,0.0


,Company,Product,perct,severity,complexity,actionability,delay,Final Risk Score
8,CAPITAL ONE FINANCIAL CORPORATION,"Money transfer, virtual currency, or money ser...",0.05,5.000000,4.000000,5.000000,0.0,2.82
12,DISCOVER BANK,Debt collection,0.10,5.000000,3.500000,5.000000,0.0,2.73
20,WELLS FARGO & COMPANY,Credit card,0.15,4.333333,4.000000,5.000000,0.0,2.71
19,WELLS FARGO & COMPANY,Checking or savings account,0.10,4.000000,4.000000,5.000000,0.0,2.63
21,WELLS FARGO & COMPANY,Credit reporting or other personal consumer re...,0.25,4.000000,3.600000,5.000000,0.0,2.60
14,JPMORGAN CHASE & CO.,Checking or savings account,0.55,4.000000,3.363636,4.818182,0.0,2.60
5,CAPITAL ONE FINANCIAL CORPORATION,Credit card,0.70,3.642857,3.214286,4.785714,0.0,2.54
15,JPMORGAN CHASE & CO.,Credit card,0.40,4.000000,3.250000,4.875000,0.0,2.54
0,"BANK OF AMERICA, NATIONAL ASSOCIATION",Checking or savings account,0.35,4.142857,3.142857,4.857143,0.0,2.53
6,CAPITAL ONE FINANCIAL CORPORATION,Credit reporting or other personal consumer re...,0.80,3.312500,3.312500,4.812500,0.0,2.53
